# CNN para Pronóstico de Series Temporales

Este notebook implementa una Red Neuronal Convolucional (CNN) aplicada al pronóstico de series temporales, inspirada en el artículo:

**Deng, Q., et al. "Short-Term Load Forecasting Based on Deep Convolutional Neural Networks and Temporal Attention Mechanism." IEEE Transactions on Smart Grid, 2020.**

En dicho artículo, los autores abordan la predicción de la demanda eléctrica a corto plazo utilizando CNN junto a un Mecanismo de Atención Temporal, logrando resultados superiores a modelos convencionales (p.ej., ARIMA, Redes Neuronales Clásicas) y a arquitecturas basadas en LSTM puras. El uso de CNN facilita la extracción de patrones locales en la serie, mientras que la atención temporal resalta secciones críticas de la ventana de entrada que resultan influyentes para la predicción final.


## Discusión del Artículo Científico

Los puntos clave que el artículo presenta son:

1. **Arquitectura CNN para datos temporales**: Normalmente, las Redes Convolucionales se han utilizado en el procesamiento de imágenes. Sin embargo, al tratar a los datos unidimensionales de una serie temporal como "secuencias" ordenadas, las convoluciones permiten capturar patrones locales y tendencias recurrentes.
2. **Mecanismo de Atención Temporal**: Integran una capa que asigna diferentes ponderaciones a cada instante de la ventana de entrada. De esta manera, el modelo presta "más atención" a aquellas partes de la secuencia que son más relevantes para la predicción.
3. **Mejor Desempeño Comparado con LSTM**: El estudio reporta menores métricas de error (RMSE, MAPE, MAE) comparado con modelos LSTM puros y combinaciones tradicionales como CNN-LSTM sin la capa de atención. El método propuesto logra capturar tanto la estacionalidad como la tendencia a corto plazo.
4. **Escalabilidad**: La arquitectura permite entrenarse con volúmenes grandes de datos, lo que la hace apta para aplicaciones industriales donde suelen abundar muestras de series históricas.

Estas consideraciones fundamentan el uso de CNN en series temporales y sirven como referencia para el ejemplo práctico que se desarrolla a continuación, utilizando datos sintéticos para ilustrar el flujo de trabajo de preparación, entrenamiento y evaluación del modelo.


## Objetivos del Notebook

1. **Demostrar la aplicación de una CNN en el pronóstico de series temporales**. Aunque se trata de datos sintéticos, la metodología es escalable y puede aplicarse a casos reales.
2. **Explorar cómo la convolución capta patrones relevantes** en ventanas de datos y mejora el rendimiento en la predicción.
3. **Resaltar la importancia de la atención temporal** (no implementada aquí de forma explícita, pero comentada en el artículo) para identificar la relevancia de diferentes partes de la secuencia.
4. **Comparar el resultado obtenido** con los puntos clave discutidos en el artículo, evidenciando la utilidad de CNN para series de tiempo.


## 1. Importación de Librerías
Se importan las librerías esenciales para el procesamiento y la construcción de la CNN.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

print(tf.__version__)


## 2. Generación de Datos Sintéticos
Para demostrar el enfoque, se creará una serie temporal con base en una onda senoidal (con ruido) simulando cierta periodicidad y variaciones aleatorias.

In [ ]:
np.random.seed(0)
# Creación del eje temporal
t = np.arange(0, 100, 0.1)
# Onda senoidal base
sinusoid = np.sin(0.1 * t)
# Ruido gaussiano
noise = np.random.normal(0, 0.1, len(t))
# Serie final
time_series = sinusoid + noise

# Visualización
plt.figure(figsize=(12, 6))
plt.plot(t, time_series)
plt.title('Serie Temporal Sintética (Seno + Ruido)')
plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.grid(True)
plt.show()

## 3. Preparación de Datos
Se construyen ventanas de 10 pasos para predecir el valor siguiente. Esta estrategia se conoce como ventana deslizante y, en problemas reales, se puede ajustar según la naturaleza de los datos (por ejemplo, en predicción de carga eléctrica a corto plazo, se elige una ventana que capture un día o una semana).

In [ ]:
n_steps = 10
X, y = [], []

for i in range(len(time_series) - n_steps):
    X.append(time_series[i : i + n_steps])
    y.append(time_series[i + n_steps])

X = np.array(X).reshape(-1, n_steps, 1)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}, y_test shape: {y_test.shape}')

## 4. Construcción del Modelo CNN
La arquitectura consta de:

- **Capa Conv1D** con 64 filtros y tamaño de kernel 3: detecta patrones locales en las secuencias.
- **MaxPooling1D**: reduce la dimensionalidad reteniendo características importantes.
- **Flatten + Capas Densas**: convierten la salida de la convolución en un vector para una regresión final.


In [ ]:
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps, 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

## 5. Entrenamiento del Modelo
Se entrena la red con 50 épocas y batch de tamaño 16. En el artículo de Deng et al., se profundiza en la técnica de early stopping y en la optimización de hiperparámetros, pero para este ejemplo se mantiene un ajuste sencillo.

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_test, y_test),
    verbose=1
)

plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Pérdida de Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de Validación')
plt.title('Historial de Entrenamiento de la CNN')
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.legend()
plt.grid(True)
plt.show()

## 6. Evaluación del Modelo
Se evalúa el desempeño calculando la raíz del error cuadrático medio (RMSE). El artículo de Deng et al. también sugiere métricas como MAE y MAPE para medir la efectividad.


In [ ]:
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE en el conjunto de prueba: {rmse:.4f}')

# Comparación gráfica
plt.figure(figsize=(12, 6))
plt.plot(y_test, label='Datos Reales', marker='o')
plt.plot(y_pred, label='Pronóstico CNN', marker='o')
plt.xlabel('Índice de Muestra')
plt.ylabel('Valor')
plt.title('Comparación de Datos Reales vs Predicción')
plt.legend()
plt.grid(True)
plt.show()

# (Opcional) Calcular MAPE
mape = np.mean(np.abs((y_test - y_pred.squeeze()) / y_test)) * 100
print(f'MAPE en el conjunto de prueba: {mape:.2f}%')

## 7. Pronósticos Futuros
Generamos 50 pasos futuros a partir de la última ventana del conjunto de prueba. En la práctica, se podrían utilizar distintas estrategias y realimentar la predicción en cada paso.


In [ ]:
future_steps = 50
future_predictions = []
last_sequence = X_test[-1].reshape(1, n_steps, 1)

for _ in range(future_steps):
    next_pred = model.predict(last_sequence)[0][0]
    future_predictions.append(next_pred)
    # Se desplaza la ventana y se incorpora la nueva predicción
    last_sequence = np.roll(last_sequence, -1, axis=1)
    last_sequence[0, -1, 0] = next_pred

plt.figure(figsize=(12, 6))
plt.plot(range(len(time_series)), time_series, label='Serie Temporal Original')
plt.plot(range(len(time_series), len(time_series) + future_steps), future_predictions, 'r--', label='Predicciones Futuras')
plt.title('Serie Temporal con 50 Predicciones Futuras')
plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.legend()
plt.grid(True)
plt.show()

## 8. Conclusiones

1. **Eficacia de la CNN**: La convolución extrae patrones locales de la serie, mejorando el pronóstico en comparación con enfoques tradicionales.
2. **Coherencia con el Artículo de Referencia**: El trabajo de Deng et al. (2020) demuestra que al combinar CNN con mecanismos de atención temporal, se pueden enfatizar los puntos más relevantes en la ventana de entrada, mejorando aún más la precisión.
3. **Resultados Obtenidos**: El RMSE y MAPE (opcionales) reflejan un error razonablemente bajo considerando el conjunto sintético. En datos reales, se esperaría una mejora más marcada al emplear más datos y técnicas de regularización.
4. **Aplicaciones Prácticas**: Se puede escalar la metodología a problemas de pronóstico de carga, demanda, ventas, entre otros, usando la misma lógica de ventanas y procesamiento.
5. **Extensiones Futuras**: Implementar capas de atención, probar con distintas arquitecturas (CNN-LSTM), e incluir otras métricas de evaluación (MAE, MAPE, sMAPE) para tener una visión más completa del desempeño.

En resumen, este enfoque basado en CNN (y potencialmente reforzado con atención temporal) brinda una alternativa sólida para el pronóstico de series temporales, alineada con las tendencias de investigación recientes y aplicable a múltiples ámbitos industriales.
